# PricePredict Debug



In [5]:
from pricepredict import PricePredict
import logging
import os
import dill

logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
# Log to a file...
logging.basicConfig(filename='dgs_pred_ui.log', level=logging.DEBUG)

def load_pp_objects():

    logger.debug("Loading PricePredict objects..,")

    sym_dpps_d_ = {}
    sym_dpps_w_ = {}

    min_dil_size = 70000
    ppo_dir = '../ppo/'

    # Find the latest .dill files in the ./ppo directory for any given symbol.
    dill_files = {}
    with os.scandir(ppo_dir) as entries:
        tot_entries = len(list(os.scandir(ppo_dir)))
        i = 0
        for entry in entries:
            if entry.is_file() and entry.name.endswith('.dill'):
                sym, period = entry.name.split('_')[:2]
                sym_period = sym + '_' + period
                # Find a key in dill_files that starts with sym and place the entry
                # the variable curr_entry.
                curr_entry = next((v for k, v in dill_files.items() if k.startswith(sym_period)), None)
                if curr_entry is not None:
                    if entry.name > curr_entry.name:
                        # Replace the entry with the newer file
                        dill_files[sym_period] = entry
                else:
                    dill_files[sym_period] = entry
            i += 1

    tot_entries = len(dill_files.keys())
    i = 0
    for sym_period in dill_files.keys():
        entry = dill_files[sym_period]
        if entry.is_file():
            sym = entry.name.split('_')[0]
            period = entry.name.split('_')[1]
            if period == 'W':
                try:
                    with open(entry, "rb") as f:
                        sym_dpps_w_[sym] = dill.load(f)
                except Exception as e:
                    logger.warning(f"Error loading PricePredict object [{sym}]: {e}")
            elif period == 'D':
                try:
                    with open(entry, "rb") as f:
                        sym_dpps_d_[sym] = dill.load(f)
                except Exception as e:
                    logger.warning(f"Error loading PricePredict object [{sym}]: {e}")
            i += 1


    return sym_dpps_d_, sym_dpps_w_


ppo_ds, ppo_ws = load_pp_objects()


DEBUG:__main__:Loading PricePredict objects..,
2024-11-24 01:59:28.969707: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-11-24 01:59:28.969726: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: motoko-7760
2024-11-24 01:59:28.969731: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: motoko-7760
2024-11-24 01:59:28.969877: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 560.35.3
2024-11-24 01:59:28.969887: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: 560.35.3
2024-11-24 01:59:28.969890: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:249] kernel version seems to match DSO: 560.35.3
